# **Caricamento e descrizione del dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trainingData = pd.read_csv('./Data/trainingData.csv')
validationData = pd.read_csv('./Data/validationData.csv')

# **Pulizia e gestione dei dati**

## Drop degli attributi superflui

In [3]:
trainingData.drop(['SPACEID','RELATIVEPOSITION','USERID','PHONEID','TIMESTAMP'], axis=1, inplace=True )
validationData.drop(['SPACEID','RELATIVEPOSITION','USERID','PHONEID','TIMESTAMP'], axis=1, inplace=True )


## Creazione attributo BUILDING_FLOOR

Per predire edificio e piano, anziché due modelli distinti, possiamo ridurre le due feature a una tramite la formula BUILDINGID*10 + FLOOR.

*Esempio:* BUILDINGID = 2, FLOOR = 3, allora BUILDING_FLOOR = 23.

In [4]:
trainingData['BUILDING_FLOOR'] = trainingData['BUILDINGID']*10 + trainingData['FLOOR']
validationData['BUILDING_FLOOR'] = validationData['BUILDINGID']*10 + validationData['FLOOR']

trainingData.drop(['FLOOR','BUILDINGID'], axis=1, inplace=True)
validationData.drop(['FLOOR','BUILDINGID'], axis=1, inplace=True)


## Divisione del dataset in features e targets

Dividiamo adesso il dataset, dividendo in X_train le colonne del dataset che contengono WAP nel nome, in y_train il restante, facciamo questa procedura sia per training set che per il test set, così da suddividere features con targets.
Avendo un unica sezione di targets, quando alleneremo il modello, ci basta specificare quale target di y_train vogliamo allenare per quel specifico modello.

In [5]:
X_train = trainingData[[i for i in trainingData.columns if 'WAP' in i]]
y_train = trainingData[[i for i in trainingData.columns if not 'WAP' in i]]
print(X_train.shape, y_train.shape)
print(y_train.columns)

(19937, 520) (19937, 3)
Index(['LONGITUDE', 'LATITUDE', 'BUILDING_FLOOR'], dtype='object')


In [6]:
X_test = validationData[[i for i in validationData.columns if 'WAP' in i]]
y_test = validationData[[i for i in validationData.columns if not 'WAP' in i]]
print(X_test.shape, y_test.shape)
print(y_test.columns)

(1111, 520) (1111, 3)
Index(['LONGITUDE', 'LATITUDE', 'BUILDING_FLOOR'], dtype='object')


## Miglioramento continuità dei dati

In [7]:
X_train = X_train.replace(to_replace=100,value=-111)
X_test = X_test.replace(to_replace=100,value=-111)

# **Allenamento del modello**

## Metodologia e librerie

Inseriamo tutte le librerie che saranno utili all'allenamento dei modelli:

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

#Cross-validation
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

#Classificatori e Regressori
from xgboost import XGBClassifier, XGBRegressor

## BUILDING_FLOOR

Definiamo innanzitutto i parametri per la cross-validation:

In [12]:
skfold = StratifiedKFold(n_splits=5, random_state=77, shuffle=True)

Costruiamo la pipeline:

In [13]:
pipeline_xgboost_c = Pipeline([
    ('data_scaling', MinMaxScaler()),
    ('feature_selection_1', VarianceThreshold()),
    ('dimension_reduction', PCA(0.85)),
    ('model', XGBClassifier())
    ])

Definiamo la griglia dei parametri ottimali da trovare:

In [14]:
param_grid_xgboost_buildingfloor = {
    "model__n_estimators": [150],
    "model__max_depth": [10],
    "model__learning_rate": [0.2]
}

Per come abbiamo costruito l'attributo, le classi non sono valori continui, per cui XGBoost non riuscirebbe ad allenarsi. Per risolvere ciò, inseriamo LabelEncoder:

In [15]:
# Codifica etichette categoriche
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train['BUILDING_FLOOR'])

Adesso possiamo creare e allenare il modello:

In [16]:
xgboost_buldingfloor = GridSearchCV(estimator = pipeline_xgboost_c,
                                param_grid = param_grid_xgboost_buildingfloor,
                                cv = skfold,
                                return_train_score=True,
                                verbose=3)
xgboost_buldingfloor.fit(X_train, y_train_encoded)
xgboost_buldingfloor.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=150;, score=(train=0.998, test=0.993) total time= 1.4min
[CV 2/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=150;, score=(train=0.998, test=0.994) total time= 1.4min
[CV 3/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=150;, score=(train=0.998, test=0.994) total time= 1.6min
[CV 4/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=150;, score=(train=0.998, test=0.990) total time= 1.7min
[CV 5/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=150;, score=(train=0.998, test=0.994) total time= 1.6min


{'model__learning_rate': 0.2,
 'model__max_depth': 10,
 'model__n_estimators': 150}

Salviamo il modello:

In [17]:
import pickle

with open('modelli/xgboost_buildingfloor.pkl','wb') as f:
        pickle.dump(xgboost_buldingfloor, f)

## LATITUDE

Definiamo i parametri della cross-validation:

In [18]:
kfold = KFold(n_splits=5, random_state=77, shuffle=True)

Costruiamo la nuova pipeline con il regressore:

In [19]:
pipeline_xgboost_r = Pipeline([
    ('data_scaling', MinMaxScaler()),
    ('feature_selection_1', VarianceThreshold()),
    ('dimension_reduction', PCA(0.85)),
    ('model', XGBRegressor())
    ])

Definiamo la griglia dei parametri ottimali:

In [20]:
param_grid_xgboost_latitude = {
    "model__n_estimators": [100],
    "model__max_depth": [10],
    "model__learning_rate": [0.2]
}

Adesso possiamo creare e allenare il modello:

In [21]:
xgboost_latitude = GridSearchCV(estimator = pipeline_xgboost_r,
                                param_grid = param_grid_xgboost_latitude,
                                cv = kfold,
                                return_train_score=True,
                                verbose=3)
xgboost_latitude.fit(X_train, y_train['LATITUDE'])
xgboost_latitude.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=100;, score=(train=0.996, test=0.992) total time= 1.6min
[CV 2/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=100;, score=(train=0.996, test=0.992) total time= 1.7min
[CV 3/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=100;, score=(train=0.996, test=0.993) total time= 1.5min
[CV 4/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=100;, score=(train=0.996, test=0.992) total time= 1.4min
[CV 5/5] END model__learning_rate=0.2, model__max_depth=10, model__n_estimators=100;, score=(train=0.997, test=0.991) total time= 1.7min


{'model__learning_rate': 0.2,
 'model__max_depth': 10,
 'model__n_estimators': 100}

Salviamo il modello:

In [22]:
with open('modelli/xgboost_latitude.pkl','wb') as f2:
        pickle.dump(xgboost_latitude, f2)

## LONGITUDE

Definiamo la griglia dei parametri ottimali

In [23]:
param_grid_xgboost_longitude = {
    "model__n_estimators": [200],
    "model__max_depth": [10],
    "model__learning_rate": [0.1]
}

Adesso possiamo creare e allenare il modello:

In [24]:
xgboost_longitude = GridSearchCV(estimator = pipeline_xgboost_r,
                                param_grid = param_grid_xgboost_longitude,
                                cv = kfold,
                                return_train_score=True,
                                verbose=3)
xgboost_longitude.fit(X_train, y_train['LONGITUDE'])
xgboost_longitude.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END model__learning_rate=0.1, model__max_depth=10, model__n_estimators=200;, score=(train=0.998, test=0.995) total time= 1.6min
[CV 2/5] END model__learning_rate=0.1, model__max_depth=10, model__n_estimators=200;, score=(train=0.998, test=0.995) total time= 1.6min
[CV 3/5] END model__learning_rate=0.1, model__max_depth=10, model__n_estimators=200;, score=(train=0.998, test=0.996) total time= 1.8min
[CV 4/5] END model__learning_rate=0.1, model__max_depth=10, model__n_estimators=200;, score=(train=0.998, test=0.996) total time= 1.8min
[CV 5/5] END model__learning_rate=0.1, model__max_depth=10, model__n_estimators=200;, score=(train=0.998, test=0.996) total time= 1.5min


{'model__learning_rate': 0.1,
 'model__max_depth': 10,
 'model__n_estimators': 200}

Salviamo il modello:

In [25]:
with open('modelli/xgboost_longitude.pkl','wb') as f3:
        pickle.dump(xgboost_longitude, f3)